In [8]:
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
import tensorflow as tf

In [6]:


base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(239, 239, 3))

x = GlobalAveragePooling2D()(base_model.output)
model = Model(inputs=base_model.input, outputs=x)

In [9]:
# Create a sequential model.
model = tf.keras.Sequential([

    # Add a dense layer with 128 units and the relu activation function.
    tf.keras.layers.Dense(128, activation='relu'),

    # Add a dense layer with 64 units and the relu activation function.
    tf.keras.layers.Dense(256, activation='relu'),

    tf.keras.layers.Dense(256, activation='relu'),

    tf.keras.layers.Dense(128, activation='relu'),

    # Add a dense layer with 1 unit and the sigmoid activation function.
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:


csv_file_path = 'relations_full_links.csv'
df = pd.read_csv(csv_file_path)

new_columns = ['color1', 'fabric1', 'category1', 'embedding1', 'color2', 'fabric2', 'category2', 'embedding2']
df_new = pd.DataFrame(columns=new_columns)

df_subset = df.iloc[0:1000]
i = 0
for index, row in df_subset.iterrows():
    try:
        img_path = '../datathon/images/' + row['image1']
        img = image.load_img(img_path, target_size=(239, 239))
        img_array = image.img_to_array(img)
        img_array = preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)
        output1 = model.predict(img_array)

        img_path = '../datathon/images/' + row['image2']
        img = image.load_img(img_path, target_size=(239, 239))
        img_array = image.img_to_array(img)
        img_array = preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)
        output2 = model.predict(img_array)
    except:
        continue
    new_row = {
        'color1': row['color1'],  # Modify or use values from the original row
        'fabric1':  row['fabric1'],  # Assign a custom value
        'category1': row['category1'],
        'embedding1': output1,
        'color2': row['color2'],  # Modify or use values from the original row
        'fabric2':  row['fabric2'],  # Assign a custom value
        'category2': row['category2'],
        'embedding2': output2,
        #'image': embeeding(row['des_filename'])  # Assign a custom value 
    }
    df_new = pd.concat([df_new, pd.DataFrame([new_row])], ignore_index=True)
    
    i += 1
    if i %20 == 0:
        print(i)

print("New DataFrame with user-defined columns:")
print(df_new.head())

1/1 [==============================] - 0s 85ms/step
20
1/1 [==============================] - 0s 89ms/step
40
1/1 [==============================] - 0s 98ms/step
60
1/1 [==============================] - 0s 100ms/step
80
1/1 [==============================] - 0s 85ms/step
100
1/1 [==============================] - 0s 95ms/step
120
1/1 [==============================] - 0s 91ms/step
140
1/1 [==============================] - 0s 92ms/step
160
1/1 [==============================] - 0s 88ms/step
180
1/1 [==============================] - 0s 88ms/step
200
1/1 [==============================] - 0s 89ms/step
220
1/1 [==============================] - 0s 99ms/step
240
1/1 [==============================] - 0s 91ms/step
260
1/1 [==============================] - 0s 95ms/step
280
1/1 [==============================] - 0s 90ms/step
300
1/1 [==============================] - 0s 88ms/step
320
1/1 [==============================] - 0s 88ms/step
340
1/1 [==============================] - 0s 93ms/step

In [17]:
from sklearn.model_selection import train_test_split

print(df_new)
X = df_new.iloc[:, :8]

y = df_new.iloc[:, 8]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

    color1 fabric1 category1  \
0        1       3         1   
1       10       4         7   
2        1       4         3   
3       10       4         3   
4        1       4         4   
..     ...     ...       ...   
995      1       4         4   
996      9       4         7   
997      4       1         1   
998      8       1         5   
999      8       4         8   

                                            embedding1 color2 fabric2  \
0    [[[[0.3635261], [0.3635261], [0.3635261], [0.3...      1       1   
1    [[[[0.3706159], [0.37139705], [0.37139705], [0...     10       4   
2    [[[[0.35642537], [0.35642537], [0.35642537], [...      1       4   
3    [[[[0.36479807], [0.3706159], [0.36983535], [0...      2       3   
4    [[[[0.35642537], [0.35642537], [0.35642537], [...      1       4   
..                                                 ...    ...     ...   
995  [[[[0.35642537], [0.35642537], [0.35642537], [...     10       4   
996  [[[[0.39314234], [0.393142

IndexError: single positional indexer is out-of-bounds

In [ ]:
# Evaluate the model.
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy of the model.
print('Accuracy:', accuracy)

In [ ]:
model.fit(X_train, y_train, epochs=50)

In [13]:
!pip3 install -U scikit-learn scipy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 361.0 kB/s eta 0:00:00MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.6/153.6 kB 1.8 MB/s eta 0:00:001.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 1.5 MB/s eta 0:00:001.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 2.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 2.8 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.1 MB/s eta 0:00:00 MB/s eta 0:00:01:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 2.3 MB/s eta 0:00:0031m4.7 MB/s eta 0:00:01
